In [1]:
########################################################################################################################
# This script organizes the feature dataset as 3D data for subsequent longitudinal-prediction modeling
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import gc
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
from time import time
from typing import List, Literal, Optional, Union
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
####################################################################################################################
# USER_SPECIFIC SETTING
# FEAT_IN_DIR_PATH: Path of the input directory of the feature datasets
# TARGET_IN_DIR_PATH: Path of the input directory storing the target datasets created in P02_Stratified_Partitioning.ipynb
####################################################################################################################
FEAT_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'
TARGET_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
OUT_DIR_PATH: str = '../00_Data/02_Processed_Data/Longitudinal_Model_Data/'

In [3]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# IMPUTE_LIST: A list of strings specifying the imputation methods. (Default: ['Zero', 'Mean', 'Median'])
# Must be a non-empty sub-list of ['Zero', 'Mean', 'Median'].
########################################################################################################################
IMPUTE_LIST: list[str] = ['Zero', 'Mean', 'Median']

In [ ]:
########################################################################################################################
# Define the partition of the datasets
########################################################################################################################
partitions: list[str] = ['train', 'test']

In [ ]:
########################################################################################################################
# Loop over all the configurations (took 21m)
########################################################################################################################
for conf_idx, (C, D, partition, impute) in enumerate(product(Cs,
                                                             Ds,
                                                             partitions,
                                                             IMPUTE_LIST), 1):
    log_head: str = f'[{conf_idx}. C={C}; D={D}; partition={partition}; impute={impute}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the latest feature datasets created in P05_Imputation.ipynb
    ####################################################################################################################
    pat_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_Patient_{partition}_v3.parquet')
    enc_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_Encounter_{partition}_v3.parquet')
    df_pat: pd.DataFrame = pd.read_parquet(pat_in_path)
    df_enc: pd.DataFrame = pd.read_parquet(enc_in_path)
    print(f'{log_head}Feature dataset loaded with dimension = {df_pat.shape}, {df_enc.shape}')

    ####################################################################################################################
    # Load the latest target dataset created in P02_Stratified_Partitioning.ipynb
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_IN_DIR_PATH, f'{C}_encounters_{D}_days_{partition}_v2.parquet')
    df_y: pd.DataFrame = pd.read_parquet(y_path)
    print(f'{log_head}Target dataset loaded with dimension = {df_y.shape}')

    ####################################################################################################################
    # Sort each dataset
    ####################################################################################################################
    id_col: str = 'PatientDurableKey'
    df_pat = df_pat.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    df_enc = df_enc.sort_values(by=[id_col, 'EncDate', 'EncounterKey'], ascending=[True, True, True]).reset_index(drop=True)
    df_y = df_y.sort_values(by=id_col, ascending=True).reset_index(drop=True)
  
    ####################################################################################################################
    # Reshape df_pat to have C copies
    ####################################################################################################################
    pat_feats: list[str] = [c for c in df_pat.columns if c != id_col]
    df_pat_feat: pd.DataFrame = df_pat[pat_feats]
    pat_feat_np: np.ndarray = df_pat_feat.to_numpy()
    pat_feat_3d: np.ndarray = np.stack([pat_feat_np] * C, axis=1)
    print(f'{log_head}Dimension of the patient-level 3D dataset: {pat_feat_3d.shape}')
    assert np.isnan(pat_feat_3d).sum() == 0
    
    ####################################################################################################################
    # Reshape df_enc to have C timestamps
    ####################################################################################################################
    assert df_enc.shape[0] == df_pat.shape[0] * C
    enc_feats: list[str] = [c for c in df_enc.columns if c not in [id_col, 'EncDate', 'EncounterKey']]
    df_enc_feat: pd.DataFrame = df_enc[enc_feats]
    enc_feat_3d: np.ndarray = df_enc_feat.to_numpy().reshape(df_pat.shape[0], C, df_enc_feat.shape[1])
    print(f'{log_head}Dimension of the encounter-level 3D dataset: {enc_feat_3d.shape}')
    assert np.isnan(enc_feat_3d).sum() == 0    

    ####################################################################################################################
    # Concatenate pat_feat_3d with enc_feat_3d
    ####################################################################################################################
    X: np.ndarray = np.concatenate([pat_feat_3d, enc_feat_3d], axis=2)
    print(f'{log_head}Concatenated dataset has a dimension = {X.shape}')

    ####################################################################################################################
    # Clean up df_y
    ####################################################################################################################
    y: np.ndarray = df_y['OutcomeLabel'].to_numpy()
    print(f'{log_head}Prevalence = {100*np.mean(y):.2f}%')

    ####################################################################################################################
    # Prepare the feature names
    ####################################################################################################################
    df_feat_out: pd.DataFrame = pd.DataFrame({'Features': pat_feats + enc_feats})

    ####################################################################################################################
    # Create a new directory and save the datasets
    ####################################################################################################################
    out_dir_s: str = os.path.join(OUT_DIR_PATH, f'{C}_encounters_{D}_days/{impute}/')
    os.makedirs(out_dir_s, exist_ok=True)
    np.save(os.path.join(out_dir_s, f'X_{partition}.npy'), X)
    np.save(os.path.join(out_dir_s, f'y_{partition}.npy'), y)
    df_feat_out.to_csv(os.path.join(out_dir_s, 'Feature_Names.csv'), index=False)
    print(f'{log_head}Packaged dataset saved in {out_dir_s}')
    print('-'*120)